# Project Natural Computing

In [ ]:
import numpy as np
from copy import deepcopy

In [ ]:
def ReLU(mat):
    return mat * (mat > 0)

def Sigmoid(mat):
    return 1/(1 + np.exp(-mat))

def SoftMax(mat):
    exp_mat = np.exp(mat)
    return exp_mat/np.sum(exp_mat, axis = 1, keepdims = True)

In [ ]:
class Network:
    def __init__(self, node_seq, activation_functions, Xavier = True):
        self.node_seq = node_seq
        self.num_layers = len(node_seq) - 1
        self.activation = activation_functions
        self.weight_mats = []
        self.bias_vecs = []
        self.fitness = 0
        for idx in range(self.num_layers):
            n_in = node_seq[idx]
            n_out = node_seq[idx + 1]
              # use Xavier initialization
            bound = np.sqrt(6/(n_in + n_out))
            if Xavier:
                self.weight_mats.append(np.random.uniform(-bound, bound, size = (n_in, n_out)))
            else: 
                self.weight_mats.append(np.zeros((n_in, n_out)))
            self.bias_vecs.append(np.zeros(n_out))
            
    def forward(self, samples):
        # note: give inputs in the form (samples, features)
        for idx in range(self.num_layers):
            samples = self.activation[idx](samples @ self.weight_mats[idx] + self.bias_vecs[idx])
        return samples
        
#net = Network([2,2,2], [ReLU, SoftMax])
#net.forward(np.array([[0,0], [0,1], [1,0], [1,1]]))[range(4), [1,0,1,0]]

In [ ]:
def cross_entropy_fitness(correct, predict):
    return max(np.sum(np.log(predict[range(predict.shape[0]), correct])) + predict.shape[0], 0)

  # computes fitness based on xor of all binary sequences of a given length
def xor_fitness(net, seq_len = 4):
    inputs = [[(num >> idx) % 2 for idx in range(seq_len)] for num in range(2**seq_len)]
    correct = [sum(seq) % 2 for seq in inputs]
    predict = net.forward(inputs)
    return cross_entropy_fitness(correct, predict)

def xor_all_fitness(net, seq_len = 4):
    inputs = [[(num >> idx) % 2 for idx in range(seq_len)] for num in range(2**seq_len)]
    correct = [sum(seq) % 2 for seq in inputs]
    predict = net.forward(inputs)
    return np.log(predict[range(predict.shape[0]), correct])

In [ ]:
def crossover_minimax(net1, net2):
    node_seq = net1.node_seq
    activation = net1.activation
    new_net1 = Network(node_seq, activation, Xavier = False)
    new_net2 = Network(node_seq, activation, Xavier = False)
    for idx in range(net1.num_layers): #perhaps random choice, instead of one net all minima and one net all maxima
        new_net1.weight_mats[idx] = np.minimum(net1.weight_mats[idx], net2.weight_mats[idx])
        new_net2.weight_mats[idx] = np.maximum(net1.weight_mats[idx], net2.weight_mats[idx])
        new_net1.bias_vecs[idx] = np.minimum(net1.bias_vecs[idx], net2.bias_vecs[idx])
        new_net2.bias_vecs[idx] = np.maximum(net1.bias_vecs[idx], net2.bias_vecs[idx])
    return new_net1, new_net2
    
def crossover_layer(net1, net2):
    node_seq = net1.node_seq
    activation = net1.activation
    new_net1 = Network(node_seq, activation, Xavier = False)
    new_net2 = Network(node_seq, activation, Xavier = False)
    select_layers = np.random.uniform(size = net1.num_layers)
    for idx in range(net1.num_layers): 
        if select_layers[idx] < 0.5:
            new_net1.weight_mats[idx] = net1.weight_mats[idx]
            new_net2.weight_mats[idx] = net2.weight_mats[idx]
            new_net1.bias_vecs[idx] = net1.bias_vecs[idx]
            new_net2.bias_vecs[idx] = net2.bias_vecs[idx]
        else:
            new_net1.weight_mats[idx] = net2.weight_mats[idx]
            new_net2.weight_mats[idx] = net1.weight_mats[idx]
            new_net1.bias_vecs[idx] = net2.bias_vecs[idx]
            new_net2.bias_vecs[idx] = net1.bias_vecs[idx]
    return new_net1, new_net2
    
def mut_layer(net):
    mut_size = np.random.uniform()
    select_layers = np.random.uniform(size = net.num_layers)
    for idx in range(net.num_layers): 
        if select_layers[idx] < 0.5:
            n_in = net.node_seq[idx]
            n_out = net.node_seq[idx + 1]
            bound = np.sqrt(6/(n_in + n_out)) * mut_size
            net.weight_mats[idx] += np.random.uniform(-bound, bound, size = (n_in, n_out))
            net.bias_vecs[idx] += np.random.uniform(-bound, bound, size = n_out)

def mut_index(net):
    mut_size = np.random.uniform()
    idx = int(np.random.choice(net.num_layers, 1))
    n_in = net.node_seq[idx]
    n_out = net.node_seq[idx + 1]
    idx2 = int(np.random.choice(n_in, 1))
    idx3 = int(np.random.choice(n_out, 1))
    bound = np.sqrt(6/(n_in + n_out)) * mut_size
    net.weight_mats[idx][idx2][idx3] += np.random.uniform(-bound, bound, size = 1)


In [ ]:
class Genetic:
    def __init__(self, pop_size, node_seq, activation_functions, fitness_function, 
                 mutation_functions, mutation_probabilities, 
                 crossover_functions, crossover_probabilities, elite):
        self.net_list = []
        self.pop_size = pop_size
        for idx in range(pop_size):
            self.net_list.append(Network(node_seq, activation_functions))
        
        self.fitness_func = fitness_function
        self.mutation_func = mutation_functions
        self.mutation_threshold = np.cumsum(mutation_probabilities)
        self.crossover_func = crossover_functions
        self.crossover_threshold = np.cumsum(crossover_probabilities)
        self.elite = elite
        self.current_best_fitness = 0
        
    def next_generation(self):
        new_net_idx = np.random.permutation(self.pop_size)
        new_net_list = [deepcopy(self.net_list[idx]) for idx in new_net_idx]
        
        select_mutate = np.random.uniform(size=self.pop_size)
        for idx in range(self.pop_size):
            for mut_idx in range(len(self.mutation_func)):
                if select_mutate[idx] < self.mutation_threshold[mut_idx]:
                    self.mutation_func[mut_idx](new_net_list[idx])
                    break
                    
        cross_net_idx = np.random.permutation(self.pop_size)
        cross_net_list = [deepcopy(self.net_list[idx]) for idx in cross_net_idx]
        select_cross = np.random.uniform(size = self.pop_size)
        for idx in range(self.pop_size//2):
            for cross_idx in range(len(self.crossover_func)):
                if select_cross[idx] < self.crossover_threshold[cross_idx]:
                    cross_net_list[2*idx], cross_net_list[2*idx + 1] =  \
                      self.crossover_func[cross_idx](cross_net_list[2*idx], \
                                                     cross_net_list[2*idx + 1])
                    
        new_net_list.extend(cross_net_list)
        new_net_list.extend(self.net_list)
        for net in new_net_list:
            net.fitness = self.fitness_func(net)
        
        new_net_list.sort(key = lambda x: -x.fitness)
        fitnesses = [net.fitness for net in new_net_list]
        probabilities = fitnesses / np.sum(fitnesses)
        net_idx = np.random.choice(len(new_net_list), self.pop_size - self.elite, p = probabilities)
        self.net_list = [new_net_list[idx] for idx in net_idx]
        self.net_list.extend(new_net_list[:self.elite])

        self.net_list.sort(key = lambda x: -x.fitness)
        if self.net_list[0].fitness > 0.0001 + self.current_best_fitness:
            print(self.net_list[0].fitness)
            self.current_best_fitness = self.net_list[0].fitness

In [ ]:
genetic_algo = Genetic(50, [4, 20, 2], [ReLU, SoftMax], xor_fitness, [mut_layer, mut_index], [0.5, 0.5], [crossover_minimax, crossover_layer], [0.5, 0.5], 1)
for gen in range(10000):
    genetic_algo.next_generation()
opt_net = genetic_algo.net_list[0]
xor_all_fitness(opt_net)
#opt_net.forward([[0,0,0,0]])